05-19-2020: Por enquanto, as tentativas de unir dados das bases de dados esta sendo difícil já que boa parte das informações de celulares não consegue ser encaixada com os seus respectivos preços. Por isso, optou-se por usar dados do site Jacotei.com,
já que ele forneceria um MVP (Mínimo Produto Viável - É uma espécie de produto simplificado e sem implementação de todas as funcionalidade para teste de mercado/hipóteses de funcionalidade/etc) e com isso um protótipo do pipeline do modelo de ML

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
import pandas as pd
import re
from datetime import datetime
import time
from selenium import webdriver
import math
import matplotlib.pyplot as plt
from unidecode import unidecode
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns


# Data extraction for Jacotei.com PART B

## Data extraction

Juntar as tabelas se mostrou infrutifero, a ideia agora é usar o jacotei...

In [2]:
def get_soup_from_url(url):
    """This function gets an url and returns a bs4 soup"""
    
    headers = {
            'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
        }
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.content)
    return soup

In [3]:
def plain_columns(dataframe):
    """This function removes accentuation from columns names and lowers letters.
    Receives a dataframe and returns the treated DataFrame"""

    
    from unidecode import unidecode
    
    unidecode_dict = {}
    for col in dataframe.columns.tolist():
        unidecode_dict[col] = unidecode(col).lower()
    dataframe.rename(columns=unidecode_dict, inplace=True)        
    
    return dataframe

In [4]:
jacotei = pd.read_csv('../pipeline/data_storage/temp/toDB2020-05-21.csv', encoding='cp1252')

In [5]:
def extract_data_from_each_smartphone_link(main_dataframe, link_column_name, backup_path):
    """This function receives a dataframe result from the extractions of the main pages of smartphones from
    maiscelular.com and extract information from each smartphone and returns it as a dataframe.
    It also saves a dataframe in a backup. It there is no backup, it starts from the beginning,
    however, in the contrary case, it loads the backup and restarts from where it stopped."""
    
    # load backup
    try:
        backup = pd.read_csv(backup_path, encoding='utf-8')
    except:
        columns = ['link']
        backup = pd.DataFrame(columns = columns)
    
    new_results = backup.copy()
    
    # check which links will be searched
    links_already_searched = backup.link.unique()
    mask = main_dataframe.loc[:,link_column_name].apply(lambda x: x not in links_already_searched)
    links_not_searched = main_dataframe.loc[mask].loc[:,link_column_name]
    
    if len(links_not_searched) == 0:
        print('There are no new smartphones to extract!')
        return backup 
    
    print(f'Extracting information from {len(links_not_searched)} new smartphones...')
    time.sleep(1)
    
    for link in tqdm(links_not_searched):
        
        try:
            #extract
            df_html = pd.read_html(link,encoding='utf-8')[0]
            #treat
            df_html = df_html.set_index(df_html.columns[0]).T
            df_html = plain_columns(df_html)
            df_html['html_error'] = False
        except:
            df_html = pd.DataFrame({'html_error': True}, index=[0])
        
        #add link
        df_html['link'] = link
        
        # add to the final result and backup
        new_results = pd.concat([new_results, df_html], ignore_index=True).reset_index(drop=True)
        new_results.to_csv(backup_path, encoding='utf-8', index=False)
    
    return new_results

In [6]:
extracted = extract_data_from_each_smartphone_link(main_dataframe=jacotei,
                                                    link_column_name = 'link_anuncio',
                                                    backup_path='jacotei_backup.csv')

Extracting information from 3 new smartphones...


100%|██████████| 3/3 [00:05<00:00,  1.75s/it]


## Merging data with original DataFrame & removing failed extracted rows

In [7]:
data = pd.merge(left=jacotei, right=extracted,left_on='link_anuncio', right_on='link')

#filtering the smartphones which the extraction did work
data = data.loc[~data.html_error]

In [8]:
backup = data.copy()

## Treating data

In [9]:
# lower all strings and remove accentuation
data = data.applymap(lambda x: unidecode(x.lower()) if type(x) == str else x)

#columns with 99% missing data and bateria em conversacao (88% missing)
delcols = ['processador','tamanho da tela','resolucao da tela','tecnologia da tela','slot de expansao',
           'marca','1x','2x','3x','4x','5x','6x','7x','8x','9x','10x','total de r$ 629,00 parcelado sem juros',
           'nome','garantia (meses)','peso produto','cest','unidade de medida','cor principal','bateria em conversacao',
           'quantidade de imeis','lastro','camada','certificado anatel','dimensoes do produto','bateria']

data = data.drop(columns=delcols)

#unused columns: they are irrelevant to the scope
data = data.drop(columns=['link','sugestoes','maior_preco','destino_do_link', 'link_anuncio', 
                          'timestamp', 'smartphone','modelo', 'operadora','html_error'])

#remove cents from prices and convert price for integer
data.loc[:, 'menor_preco'] = data.loc[:, 'menor_preco'].apply(lambda x: x.split(',')[0].split('r$')[1].strip())
data.loc[:, 'menor_preco'] = data.loc[:, 'menor_preco'].apply(lambda x: int(x.split('.')[0] + x.split('.')[1]) if '.' in x else int(x))

#change all commas to dots
data = data.applymap(lambda x: x.replace(',','.') if ',' in str(x) else x)

#mudar parametros numericos (com unidades) para floats apenas
# tela:polegada,camera traseira:megapixels, camera selfie:megapixels, 
# altura:cm, largura:cm, profundidade:cm, memoria interna:GB, peso:g
colunas_numéricas = ['tela', 'camera traseira', 'camera selfie', 'altura', 'largura', 'profundidade', 'memoria interna', 'peso']

#recording columns of memoria interna which are in MB and change them later to GB
mask_megabytes = data.loc[:,'memoria interna'].apply(lambda x: 'MB' in str(x))
data.loc[:,colunas_numéricas] = data.loc[:,colunas_numéricas].applymap(lambda x: float(re.sub('[^0-9.\-]','',x)) if type(x) == str else x)

#converting MG to GB
data.loc[mask_megabytes,'memoria interna'] = data.loc[mask_megabytes,'memoria interna']/1024

#simplifying the 'data_anuncio': getting only the years
data.loc[:,'data_anuncio'] = data.loc[:,'data_anuncio'].apply(lambda x: int(x.split('/')[1]))

# replace info in sistema operacional: x for 10
data.loc[:,'sistema operacional'] = data.loc[:,'sistema operacional'].apply(lambda x: x.replace('x','10') if 'x' in str(x) else x)

**arranging operational system**

In [10]:
def osys(x, os):
    
    if os in str(x):
        try:
            return float(re.sub('[^0-9.\-]','',x))
        except:
            if 'oreo' in str(x):
                return 8.0
    else:
        return 0

In [11]:
# adding operational system
data['android'] = data.loc[:,'sistema operacional'].apply(lambda x: osys(x,'android'))
data['ios'] = data.loc[:,'sistema operacional'].apply(lambda x: osys(x,'ios'))
data['wind'] = data.loc[:,'sistema operacional'].apply(lambda x: osys(x,'wind'))
data = data.drop(columns='sistema operacional')

**Changing Yes for 1 and No for zero**

In [12]:
#changing 'Sim' for 1. Notice! NaN were converted to zero!
data.iloc[:,[4, 5, 6, 9, 13,18]] = data.iloc[:,[4, 5, 6, 9, 13,18]].applymap(lambda x: 1 if 'sim' in str(x) else 0)

**removal of 'conexoes', 'funcoes'**

In [13]:
#those columns will be removed for now but can be used to generate more info later
data = data.drop(columns = ['conexoes', 'funcoes'])

**Removal of 'altura' and 'largura' (both 20% missing)**
They are correlated to the size of the screen (tela)

In [14]:
data = data.drop(columns = ['altura', 'largura'])

### **Feature Generation: Let's check some variables that can be kept, modified or created**

#### Generating variable: brand of smartphones (iphone, samsung, huawei, other_brand)

This was based on the top 10 brands in the brazilian market (as of 2020).

In [15]:
def brandfilter(x):
    """Receives a string of device information and returns a string related to the device brand that device belongs"""
    
    
    topmarcas = ['samsung', 'apple', 'motorola', 'asus', 'lg', 'huawei', 'xiaomi', 'sony', 'lg', 'positivo', 'nokia']
    
    if 'iphone' in x:
        return 'apple'
    
    else:
        for marca in topmarcas:
            if marca in x:
                return marca
        
    return 'other'

In [16]:
data['brand'] = data.loc[:,'modelo_celular'].apply(lambda x: brandfilter(x))

#### *'recursos da tela'* ~60% missing

são propriedades de um celular, é de se esperar que o nan seja que não foi importande cadastrar features na pagina do smartphone talvez?


[nan, 'full hd', "resistente a respingos d'agua",
'auto rotacao. full hd. multitouch', 'auto rotacao',
"auto rotacao. multitouch. resistente a respingos d'agua",
'force touch', 'curva', "full hd. resistente a respingos d'agua",
'auto rotacao. multitouch. resistente a arranhoes',
'resistente a arranhoes', 'auto rotacao. full hd', 'multitouch',
'resistente a queda', '3d']

In [17]:
data.loc[:,'recursos da tela'] = data.loc[:,'recursos da tela'].fillna('sem_recurso')

In [18]:
features = ['sem_recurso','auto rotacao', 'multitouch', "resistente a respingos d'agua", 'force touch', 'curva', 'full hd', 'multitouch',
'resistente a arranhoes', 'resistente a queda', '3d']

In [19]:
for feature in features:
    data[feature] = data.loc[:,'recursos da tela'].apply(lambda x: 1 if feature in x else 0)

In [20]:
data = data.drop(columns='recursos da tela')

#### Peso: fill with mean of each year
Fill up NaNs based on the mean weight of each anuncio_year

In [21]:
for year in data.data_anuncio.unique():
    year_mask = data.data_anuncio == year
    mean_of_year = data.loc[year_mask].peso.mean()
    data.loc[year_mask, 'peso'] = data.loc[year_mask, 'peso'].fillna(mean_of_year)

#### Profundidade: fill with mean of each year
Fill up NaNs based on the mean width of each anuncio_year

In [22]:
for year in data.data_anuncio.unique():
    year_mask = data.data_anuncio == year
    mean_of_year = data.loc[year_mask].profundidade.mean()
    data.loc[year_mask, 'profundidade'] = data.loc[year_mask, 'profundidade'].fillna(mean_of_year)

#### Cameras: fill with zero

We will assume that the capabilities of the cameras that were not found (NaN) can be considered that the device does not has a camera:

In [23]:
data.loc[:,['camera traseira','camera selfie']] = data.loc[:,['camera traseira','camera selfie']].fillna(0)

#### **Other missing**

Here the approach would be to fill the missing values with 'not_informed'. The idea is that if the jacotei team did not fill certain information about a smartphone, that information is not that valuable for that smartphone...

**Fill tecnologia with 'tec_not_informed'**

In [24]:
data.loc[:, 'tecnologia'] = data.loc[:, 'tecnologia'].fillna('tec_not_informed')

**Fill 'gravacao de video' with 'rec_capacity_not_informed'

In [25]:
data.loc[:,'gravacao de video'] = data.loc[:,'gravacao de video'].fillna('rec_capacity_not_informed')

**Fill 'memoria interna' based on 'modelo_celular**

Some smartphone title (modelo_celular) have the amount of internal memmory which can be caught and used to fill the 'memoria interna' column

In [26]:
def memmory(x):
    
    if 'gb' in x:
        return int(re.findall('\d+gb',x)[0].split('gb')[0])
        
    else:
        return np.nan

In [27]:
nan_mask = data.loc[:,'memoria interna'].isna()
data.loc[nan_mask,'memoria interna'] = data.loc[nan_mask,'modelo_celular'].apply(lambda x:  memmory(x))

#### Final check

**Dropping any columns that has more than 95% missing**

In [28]:
data = data.drop(columns=data.loc[:,data.isna().sum()/data.shape[0]*100 > 95].columns)

In [29]:
data.formato.unique()

array(['barra', 'flip', nan, 'slider'], dtype=object)

In [30]:
columns_with_nan = data.loc[:, data.isna().sum()/data.shape[0] > 0].columns
print('Percentage of Missing values: \n')
data[columns_with_nan].isna().sum()/data.shape[0]*100

Percentage of Missing values: 



tela               3.594771
formato            1.225490
memoria interna    6.372549
dtype: float64

**Adding the links back**

In [31]:
backup.loc[:,['link_anuncio']].head().reset_index()

,index,link_anuncio
0,0,https://www.jacotei.com.br/smartphone-samsung-...
1,1,https://www.jacotei.com.br/smartphone-huawei-p...
2,13,https://www.jacotei.com.br/iphone-11-pro-max-a...
3,14,https://www.jacotei.com.br/iphone-x-apple-256g...
4,18,https://www.jacotei.com.br/iphone-11-pro-max-a...


In [32]:
results = pd.merge(data,backup.loc[:,['link_anuncio']],left_index=True, right_index=True)

**Storing changes**

In [33]:
results.to_csv('devices.csv')